# Part 1: Introduction to Tensors

In [ ]:
import Base:println,+

mutable struct Tensor
    data 
end

+(a::Tensor, b::Tensor) = a.data + b.data

println(t::Tensor) = println(t.data)
    
x = Tensor([1,2,3,4,5])
print(x)

y = x + x
print(y)

In [ ]:
function workspace()
   atexit() do
       run(`$(Base.julia_cmd())`)
   end
   exit()
end

In [ ]:
workspace()

# Part 2: Introduction to Autograd

In [ ]:
import Base:println,+

mutable struct Tensor
    data
    creators
    creation_op
    grad 
    Tensor(data; creators=nothing, creation_op = nothing) = 
    new(data, creators, creation_op)
end

function backward(t::Tensor, grad)
    t.grad = grad
    
    if t.creation_op == "add"
        backward(t.creators[1], grad)
        backward(t.creators[2], grad)
    end
end

+(a::Tensor, b::Tensor) = Tensor(a.data + b.data; creators=[a,b], creation_op="add")
println(t::Tensor) = println(t.data)
println(t::Array{Tensor,1}) = println([i.data for i in t])
    
x = Tensor([1,2,3,4,5])
y = Tensor([2,2,2,2,2])

z = x + y
backward(z, Tensor([1,1,1,1,1]))

In [ ]:
println(x.grad)
println(y.grad)
println(z.creators)
println(z.creation_op)

In [ ]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])
d = Tensor([-1,-2,-3,-4,-5])

e = a + b
f = c + d
g = e + f

backward(g, Tensor([1,1,1,1,1]))

println(a.grad)

# Part 3: Tensors That Are Used Multiple Times

In [ ]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])

d = a + b
e = b + c
f = d + e
backward(f, Tensor([1,1,1,1,1]))

b.grad.data == [2,2,2,2,2]

In [ ]:
b.grad.data

# Part 4: Upgrading Autograd to Support Multiple Tensors

In [ ]:
using Random
import Base:+,println

mutable struct Tensor
    data
    autograd
    creators
    creation_op
    id
    children
    grad 
    function Tensor(data; autograd=false, creators=nothing, creation_op = nothing, id=nothing)
        if isnothing(id)
            id = rand(1:100000)
        end
        T = new(data, autograd, creators, creation_op, id)
        T.children = Dict()
        T.grad = nothing
        
        if !(isnothing(creators))
            for c in creators
                if haskey(c.children, T.id)
                    c.children[T.id] += 1
                else
                    c.children[T.id] = 1
                end
            end
        end
        return T
    end
end

function all_children_grads_accounted_for(t::Tensor)
    for (id, cnt) in t.children
        if (cnt != 0)
            return false
        end
    end
    return true
end

function backward(t::Tensor, grad=nothing, grad_origin=nothing)
    if t.autograd
        grad = Tensor(ones(size(t.data)))
    
        if !(isnothing(grad_origin))
            if t.children[grad_origin.id] == 0
                throw("cannot backprop more than once")
            else
                t.children[grad_origin.id] -= 1
            end
        end
        
        if isnothing(t.grad)
            t.grad = grad
        else
            t.grad += grad
        end
        
        # grads must not have grads of their own
        @assert !grad.autograd
        
        # only continue backpropping if there's something to
        # backprop into and if all gradients (from children)
        # are accounted for override waiting for children if
        # "backprop" was called on this variable directly
        
        if (!isnothing(t.creators) && (all_children_grads_accounted_for(t) || isnothing(grad_origin)))
            if t.creation_op == "add"
                backward(t.creators[1], t.grad, t)
                backward(t.creators[2], t.grad, t)
            end
        end
    end
end

function +(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data .+ b.data; autograd=true, creators=[a,b], creation_op = "add")
    end
    return Tensor(a.data+b.data)
end

println(t::Tensor) = println(t.data)

a = Tensor([1,2,3,4,5]; autograd=true)
b = Tensor([2,2,2,2,2]; autograd=true)
c = Tensor([5,4,3,2,1]; autograd=true)

d = a + b
e = b + c
f = d + e

backward(f, Tensor([1,1,1,1,1]))

println(b.grad.data == [2,2,2,2,2])

# Part 5: Add Support for Negation

In [ ]:
using Random
import Base:+,-,println

mutable struct Tensor
    data
    autograd
    creators
    creation_op
    id
    children
    grad 
    function Tensor(data; autograd=false, creators=nothing, creation_op = nothing, id=nothing)
        if isnothing(id)
            id = rand(1:100000)
        end
        T = new(data, autograd, creators, creation_op, id)
        T.children = Dict()
        T.grad = nothing
        
        if !(isnothing(creators))
            for c in creators
                if haskey(c.children, T.id)
                    c.children[T.id] += 1
                else
                    c.children[T.id] = 1
                end
            end
        end
        return T
    end
end

function all_children_grads_accounted_for(t::Tensor)
    for (id, cnt) in t.children
        if (cnt != 0)
            return false
        end
    end
    return true
end

function backward(t::Tensor, grad=nothing, grad_origin=nothing)
    if t.autograd
        if isnothing(grad)
            grad = Tensor(ones(size(t.data)))
        end
    
        if !(isnothing(grad_origin))
            if t.children[grad_origin.id] == 0
                throw("cannot backprop more than once")
            else
                t.children[grad_origin.id] -= 1
            end
        end
        
        if isnothing(t.grad)
            t.grad = grad
        else
            t.grad += grad
        end
        
        # grads must not have grads of their own
        @assert !grad.autograd
        
        # only continue backpropping if there's something to
        # backprop into and if all gradients (from children)
        # are accounted for override waiting for children if
        # "backprop" was called on this variable directly
        
        if (!isnothing(t.creators) && (all_children_grads_accounted_for(t) || isnothing(grad_origin)))
            if t.creation_op == "add"
                backward(t.creators[1], t.grad, t)
                backward(t.creators[2], t.grad, t)
            end
            if t.creation_op == "neg"
                backward(t.creators[1], -t.grad)
            end
        end
    end
end

function +(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data .+ b.data; autograd=true, creators=[a,b], creation_op = "add")
    end
    return Tensor(a.data+b.data)
end

function -(a::Tensor)
    if (a.autograd)
        return Tensor(a.data .* -1; autograd=true, creators=[a], creation_op = "neg")
    end
    return Tensor(a.data .* -1)
end


println(t::Tensor) = println(t.data)

a = Tensor([1,2,3,4,5]; autograd=true)
b = Tensor([2,2,2,2,2]; autograd=true)
c = Tensor([5,4,3,2,1]; autograd=true)

d = a + (-b)
e = (-b) + c
f = d + e

backward(f, Tensor([1,1,1,1,1]))

print(b.grad.data == [-2,-2,-2,-2,-2])

# Part 6: Add Support for Additional Functions

In [2]:
using Random
import Base:+,-,*,println, sum, broadcasted, size, adjoint, display

mutable struct Tensor
    data
    autograd
    creators
    creation_op
    id
    children
    grad 
    function Tensor(data; autograd=false, creators=nothing, creation_op = nothing, id=nothing)
        if isnothing(id)
            id = rand(1:100000)
        end
        T = new(data, autograd, creators, creation_op, id)
        T.children = Dict()
        T.grad = nothing
        
        if !(isnothing(creators))
            for c in creators
                if haskey(c.children, T.id)
                    c.children[T.id] += 1
                else
                    c.children[T.id] = 1
                end
            end
        end
        return T
    end
end

function all_children_grads_accounted_for(t::Tensor)
    for (id, cnt) in t.children
        if (cnt != 0)
            return false
        end
    end
    return true
end

function backward(t::Tensor, grad=nothing, grad_origin=nothing)
    if t.autograd
        if isnothing(grad)
            grad = Tensor(ones(size(t.data)))
        end
    
        if !(isnothing(grad_origin))
            if t.children[grad_origin.id] == 0
                throw("cannot backprop more than once")
            else
                t.children[grad_origin.id] -= 1
            end
        end
        
        if isnothing(t.grad)
            t.grad = grad
        else
            t.grad += grad
        end
        
        # grads must not have grads of their own
        @assert !grad.autograd
        
        # only continue backpropping if there's something to
        # backprop into and if all gradients (from children)
        # are accounted for override waiting for children if
        # "backprop" was called on this variable directly
        
        if (!isnothing(t.creators) && (all_children_grads_accounted_for(t) || isnothing(grad_origin)))
            if t.creation_op == "add"
                println("add on $(t.id)")
                backward(t.creators[1], t.grad, t)
                backward(t.creators[2], t.grad, t)
            end
            
            if t.creation_op == "sub"
                println("sub on $(t.id)")
                backward(t.creators[1], t.grad, t)
                backward(t.creators[2], -t.grad, t)
            end
            
            if t.creation_op == "mul"
                println("mul on $(t.id)")
                new_ = t.grad .* t.creators[2]
                backward(t.creators[1], new_, t)
                new_ = t.grad .* t.creators[1]
                backward(t.creators[2], new_, t)
            end
            
            if t.creation_op == "mm"
                println("mm on $(t.id)")
                c1 = t.creators[1]
                c2 = t.creators[2]
                new_ =  t.grad * c2' ################
                backward(c1, new_)
                new_ = c1' * t.grad
                backward(c2, new_)
            end
                  
            if t.creation_op == "transpose"
                println("transpose on $(t.id)")
                backward(t.creators[1], t.grad')
            end
            
            if occursin("sum", t.creation_op)
                println("sum on $(t.id)")
                dim = parse(Int, split(t.creation_op, "_")[2])
                backward(t.creators[1], expand(t.grad, dim, size(t.creators[1].data)[dim]))
            end
            
            if occursin("expand", t.creation_op)
                println("expand on $(t.id)")
                dim = parse(Int, split(t.creation_op, "_")[2])
                backward(t.creators[1], sum(t.grad;dims=dim))
            end              
            
            if t.creation_op == "neg"
                println("neg")
                backward(t.creators[1], -t.grad)
            end
        end
    end
end

size(a::Tensor) = size(a.data)

function +(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data + b.data; autograd=true, creators=[a,b], creation_op = "add")
    end
    return Tensor(a.data+b.data)
end

function -(a::Tensor)
    if (a.autograd)
        return Tensor(a.data .* -1; autograd=true, creators=[a], creation_op = "neg")
    end
    return Tensor(a.data .* -1)
end

function -(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data - b.data; autograd=true, creators=[a,b], creation_op = "sub")
    end
    return Tensor(a.data-b.data)
end

#element-wise multiplication
function broadcasted(*, a::Tensor, b::Tensor)
    new_data = a.data
    for i=1:length(new_data)
#         println(b.data[i],"  ", new_data[i])
        new_data[i] *= b.data[i]
    end
    if (a.autograd && b.autograd)
        return Tensor(new_data; autograd=true, creators=[a,b], creation_op ="mul")
    end
    return Tensor(new_data)
end

function sum(a::Tensor; dims=1)
    new_ = dropdims(sum(a.data ;dims=dims), dims = tuple(findall(size(a) .== 1)...))
    if (a.autograd && b.autograd)
        return Tensor(new_; autograd=true, creators=[a], creation_op = "sum_"*string(dims))
    end
    return Tensor(sum(new_;dims=dims))
end

function expand(a::Tensor, dim, copies)
    sz = size(a)
    rep = ntuple(d->d==dim ? copies : 1, length(sz)+1)
    new_size = ntuple(d->d<dim ? sz[d] : d == dim ? 1 : sz[d-1], length(sz)+1)
    new_data =  repeat(reshape(a.data, new_size), outer=rep)
    if (a.autograd)
        return Tensor(new_data; autograd=true, creators=[a], creation_op = "expand_"*string(dim))
    end
    return Tensor(new_data)
end

#transpose
function adjoint(a::Tensor)
    if (a.autograd)
        return Tensor(a.data';autograd=true, creators=[a], creation_op = "transpose")
    end
    return Tensor(a.data')
end

#matrix multiply 
function *(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data * b.data; autograd=true, creators=[a,b], creation_op = "mm")
    end
    return Tensor(a.data * b.data)
end

    
println(t::Tensor) = println(t.data)
display(t::Tensor) = display(t.data)

a = Tensor([1,2,3,4,5]; autograd=true)
b = Tensor([2,2,2,2,2]; autograd=true)
c = Tensor([5,4,3,2,1]; autograd=true)

d = a + (-b)
e = (-b) + c
f = d + e

backward(f, Tensor([1,1,1,1,1]))

print(b.grad.data == [-2,-2,-2,-2,-2])

add on 33852
add on 1859
neg
add on 92377
neg
true

In [ ]:
x = Tensor([1 2 3;4 5 6];autograd=true)

In [ ]:
sum(x;dims=2).data

In [ ]:
x_expand = expand(x,3,4).data

In [ ]:
x_expand[1,:,:]

In [ ]:
x_expand[2,:,:]

# Part 7: Use Autograd to Train a Neural Network

#### Previously we would train a model like this

In [4]:
using Random: seed!
seed!(0)

data = [ 0 0; 0 1; 1 0; 1 1;]
target = [0; 1; 0; 1]

weights_0_1 = rand(2,3)
weights_1_2 = rand(3,1)

for i=1:10
    
#     # Predict
    layer_1 = data * weights_0_1
    layer_2 = layer_1 * weights_1_2
    
#     # Compare
    diff = (layer_2 - target)
    sqdiff = (diff .* diff)
    loss = sum(sqdiff;dims=1) # mean squared error loss

#     # Learn: this is the backpropagation piece
    layer_1_grad = diff * weights_1_2'
    weight_1_2_update = layer_1' * diff
    weight_0_1_update = data' * layer_1_grad
    
    weights_1_2 .-= weight_1_2_update .* 0.1
    weights_0_1 .-= weight_0_1_update .* 0.1
    println(loss[1])
end

1.319677517363771
0.645370221071993
0.48046558059136113
0.42673209144608215
0.3930618130835104
0.363594896295545
0.33605797739677096
0.3100545140899324
0.28544274299374384
0.2621351937537131


In [ ]:
using Random: seed!
seed!(0)

data = Tensor([ 0 0; 0 1; 1 0; 1 1;], autograd=true)
target = Tensor([0; 1; 0; 1], autograd=true)

w = []
push!(w, Tensor(rand(2,3), autograd=true))
push!(w, Tensor(rand(3,1), autograd=true))

for i=1:10

#     # Predict
    pred_1 = data * w[1]
    pred_2 = pred_1 * w[2]
    diff_1 = pred_2 .- target
    diff_2 = diff_1 .* diff_1
    
#     # Compare
    loss = sum(diff_2;dims=1)
    
#     # Learn
    backward(loss, Tensor(ones(Float32, size(loss.data))))

    for w_ in w
        w_.data .-= w_.grad.data .* 0.1
        w_.grad.data .*= 0
    end

    println(loss)
end